In [81]:
import numpy as np
import math
import tensorflow as tf
%load_ext tensorboard

In [2]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [105]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    with tf.GradientTape() as tape:
        tape.watch(raw_u0)
        bzero = tf.zeros([inFilt.shape[4]])
        u0 = TFconv(raw_u0, inFilt[0], bzero)
        
        loss = tf.math.square(u0 - raw_u0)
        [dl_dpFilt, dl_dinFilt, dl_db] = tape.gradient(loss, [pFilt, inFilt, b])
        print("dl_dpFilt: ", dl_dpFilt)
        print("dl_dinFilt: ", dl_dinFilt)
        print("dl_db: ", dl_db)
        for q in tape.watched_variables():
            print(q.name)
        
        #for loop begins, h=0
        x0 = tf.nn.leaky_relu(u0, alpha=0.1)
        zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
        #print(zInt0)
        z0 = trapezoidIntegral(zInt0, ds)
        u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
        #h=1
        x1 = tf.nn.leaky_relu(u1, alpha=0.1)
        zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
        zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
        #zInt1 = zInt1_0 + zInt1_1
        zInt1 = tf.stack([zInt1_0, zInt1_1])
        z1 = trapezoidIntegral(zInt1, ds)
        u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
        #h=2
        x2 = tf.nn.leaky_relu(u2, alpha=0.1)#
        zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
        zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
        zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
        #zInt2 = zInt2_0 + zInt2_1 + zInt2_2
        zInt2 = tf.stack([zInt2_0, zInt2_1, zInt2_2])
        z2 = trapezoidIntegral(zInt2, ds)#
        u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
        #h=3
        x3 = tf.nn.leaky_relu(u3, alpha=0.1)
        zInt3_0 = TFconv(x0, pFilt[0,3], b[3] / ((3+1)*ds)) #s=0
        zInt3_1 = TFconv(x1, pFilt[1,3], b[3] / ((3+1)*ds)) #s=1
        zInt3_2 = TFconv(x2, pFilt[2,3], b[3] / ((3+1)*ds)) #s=2
        zInt3_3 = TFconv(x3, pFilt[3,3], b[3] / ((3+1)*ds)) #s=3
        #zInt3 = zInt3_0 + zInt3_1 + zInt3_2 + zInt3_3
        zInt3 = tf.stack([zInt3_0, zInt3_1, zInt3_2, zInt3_3])
        z3 = trapezoidIntegral(zInt3, ds)
        u4 = z3 + TFconv(raw_u0, inFilt[4], bzero)
        #end loop
        x4 = tf.nn.leaky_relu(u4, alpha=0.1)

        #print("z with h=0", z0)
        #print("z with h=1", z1)
        #print("z with h=2", z2)
        #print("z with h=3", z3)

        x = tf.stack([x0,x1,x2,x3,x4])
        u = tf.stack([u0,u1,u2,u3,u4])
    
    return x,u

In [145]:
def loopTFuSoln(raw_u0, ds, inFilt, pFilt, b, x, u):
    zInt = None
    t1 = inFilt.shape[0]
    input_shape = raw_u0.shape
    bzero = tf.zeros([inFilt.shape[4]])
    
    with tf.GradientTape() as tape:
        tape.watch(raw_u0)
        u[0].assign(TFconv(raw_u0, inFilt[0], bzero))

        loss = tf.math.square(u[0] - raw_u0)
        [dl_dpFilt, dl_dinFilt, dl_db] = tape.gradient(loss, [pFilt, inFilt, b])
        print("dl_dpFilt: ", dl_dpFilt)
        print("dl_dinFilt: ", dl_dinFilt)
        print("dl_db: ", dl_db)

        for h in range(t1-1):
            x[h].assign(tf.nn.leaky_relu(u[h], alpha=0.1))
            if (zInt is None):
                zInt = tf.Variable(tf.zeros((h+1,*input_shape)))
            for s in range(h+1):
                zInt[s].assign(TFconv(x[s], pFilt[s,h], b[h] / ((h+1)*ds)))
            z = trapezoidIntegral(zInt, ds)
            u[h+1].assign(z + TFconv(raw_u0, inFilt[h+1], bzero))
        x[-1].assign(tf.nn.leaky_relu(u[-1], alpha=0.1))
        
        #loss = tf.math.square(u[-1] - raw_u0)
        #print("Loss: ", tf.math.reduce_sum(loss))
        #[dl_dpFilt, dl_dinFilt, dl_db] = tape.gradient(loss, [pFilt, inFilt, b])
        #print("dl_db: ", dl_db)
        #print(tf.transpose(dl_dpFilt, perm=[0,1,4,5,2,3]))
    return x,u 

In [97]:
t0 = 4
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

np.set_printoptions(suppress=True)
rng = np.random.default_rng(42)
uIn = rng.standard_normal(input_shape, dtype=np.float32)
iFilt = rng.standard_normal((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.standard_normal((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.standard_normal((t1,1), dtype=np.float32)

#xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
#print(xOut)

In [148]:
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)

TFiFiltTensor = tf.transpose(tf.compat.v1.get_variable("iFiltTensor", initializer=iFilt), perm=[0,3,4,1,2])
TFiFilt = tf.Variable(TFiFiltTensor, name="iFilt")

TFpFiltTensor = tf.transpose(tf.compat.v1.get_variable("pFiltTensor", initializer=paramFilt), perm=[0,1,4,5,2,3])
TFpFilt = tf.Variable(TFpFiltTensor, name="pFilt")

TFbiasVec = tf.compat.v1.get_variable("biasVec", initializer=biasVec)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(TFuOut, perm=[4,0,1,2,3]))

dl_dpFilt:  None
dl_dinFilt:  tf.Tensor(
[[[[[ 31.10378 ]]

   [[ 42.61736 ]]

   [[-55.96108 ]]]


  [[[ 21.822216]]

   [[-89.51662 ]]

   [[-21.495705]]]


  [[[-27.936436]]

   [[ 15.755135]]

   [[ 32.94056 ]]]]



 [[[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]]



 [[[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]]



 [[[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]]



 [[[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]


  [[[  0.      ]]

   [[  0.      ]]

   [[  0.      ]]]]], shape=(5, 3, 3, 1, 1), dtype=float32)
dl_db:  None
iFilt:0
tf.Ten

In [146]:
input_shape = TFuIn.shape
u = tf.Variable(tf.zeros((t1,*input_shape)), name="u")
x = tf.Variable(tf.zeros((t1,*input_shape)), name="x")

#loopTFxOut, loopTFuOut = loopTFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec, x, u)
#print(tf.transpose(loopTFuOut, perm=[4,0,1,2,3]))
TFloopUSoln = tf.function(loopTFuSoln)
tf.config.run_functions_eagerly(False)
TFloopUSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec, x, u)

dl_dpFilt:  None
dl_dinFilt:  None
dl_db:  None


ValueError: in user code:

    <ipython-input-145-2e1b95317a97>:22 loopTFuSoln  *
        zInt[s].assign(TFconv(x[s], pFilt[s,h], b[h] / ((h+1)*ds)))
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1293 _SliceHelperVar
        return _slice_helper(var.value(), slice_spec, var)
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1036 _slice_helper
        return strided_slice(
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1209 strided_slice
        op = gen_array_ops.strided_slice(
    /Users/eyloncaplan/opt/anaconda3/lib/python3.8/site-pa